1. Написать теггер на данных с русским языком

- проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
- написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
- сравнить все реализованные методы, сделать выводы

2. Проверить, насколько хорошо работает NER
- проверить NER из nltk/spacy/deeppavlov.
- написать свой NER, попробовать разные подходы.
    - передаём в сетку токен и его соседей.
    - передаём в сетку только токен.
    - свой вариант.
- сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


In [1]:
import re
import corus
import pyconll
import nltk
import spacy
import os
import matplotlib
import pandas as pd
import numpy as np
import tensorflow as tf
from spacy import displacy
from corus import load_ne5
from sklearn import model_selection, preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from nltk.tokenize import word_tokenize
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger, RegexpTagger
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from razdel import tokenize
import warnings


warnings.filterwarnings('ignore')
data_path = r'/media/dmitriy/Disk/Downloads/ai_nlp_hw_data/hw_5/'

2023-12-17 00:26:44.507338: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 00:26:44.507377: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 00:26:44.507413: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-17 00:26:44.516688: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Задание 1.

In [2]:
data_train = pyconll.load_from_file(os.path.join(data_path, 'ru_syntagrus-ud-train.conllu'))
data_test = pyconll.load_from_file(os.path.join(data_path, 'ru_syntagrus-ud-dev.conllu'))

In [3]:
fdata_train = []
for sent in data_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
fdata_test = []
for sent in data_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
fdata_sent_test = []
for sent in data_test[:]:
    fdata_sent_test.append([token.form for token in sent])

Проверка работы теггеров.

In [4]:
default_tagger = nltk.DefaultTagger('NN')
default_acc = default_tagger.evaluate(fdata_test)

unigram_tagger = UnigramTagger(fdata_train)
unigram_acc = unigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train)
bigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train)
trigram_acc = trigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_unigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_bigram_unigram_acc = trigram_tagger.evaluate(fdata_test)

print(f'Accuracy:\nDefault Tagger: {round(default_acc, 3)},\nUnigram Tagger: {round(unigram_acc, 3)},\nBigram Tagger: {round(bigram_acc, 5)},\n'
      f'Trigram Tagger: {round(trigram_acc, 3)},\nBigram and Unigram Tagger: {round(bigram_unigram_acc, 5)},\n'
      f'Trigram, Bigram and Unigram Tagger: {round(trigram_bigram_unigram_acc, 5)},\n')

Accuracy:
Default Tagger: 0.0,
Unigram Tagger: 0.824,
Bigram Tagger: 0.60939,
Trigram Tagger: 0.178,
Bigram and Unigram Tagger: 0.82928,
Trigram, Bigram and Unigram Tagger: 0.82914,



Свой теггер. Преобразование датасета в списки солв и POS-разметки.

In [5]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(' ' if tok[0] is None else tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [6]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [7]:
test_enc_labels = le.transform(test_label)

In [8]:
vectorizers = [CountVectorizer(ngram_range=(1, 5), analyzer='char'), 
               TfidfVectorizer(ngram_range=(1, 5), analyzer='char'), 
               HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=1000)] 
vectorizers_word = [CountVectorizer(ngram_range=(1, 5), analyzer='word'), 
               TfidfVectorizer(ngram_range=(1, 5), analyzer='word'), 
               HashingVectorizer(ngram_range=(1, 5), analyzer='word', n_features=1000)] 
n_features = [2000, 3000, 5000]
vectorizers_hash = [HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=feat) for feat in n_features]
vectorizers_hash_word = [HashingVectorizer(ngram_range=(1, 5), analyzer='word', n_features=feat) for feat in n_features]
f1_scores = []
accuracy_scores = []

for vectorizer in vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok)
    
    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    f1 = f1_score(test_enc_labels, pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(test_enc_labels, pred)
    accuracy_scores.append(acc)
    
    print(vectorizer)
    print(classification_report(test_enc_labels, pred, target_names=le.classes_))

CountVectorizer(analyzer='char', ngram_range=(1, 5))
              precision    recall  f1-score   support

         ADJ       0.93      0.93      0.93     15103
         ADP       0.98      1.00      0.99     13717
         ADV       0.92      0.92      0.92      7783
         AUX       0.82      0.96      0.88      1390
       CCONJ       0.89      0.97      0.93      5672
         DET       0.89      0.70      0.79      4265
        INTJ       0.39      0.29      0.33        24
        NOUN       0.94      0.97      0.96     36238
      NO_TAG       1.00      0.77      0.87       265
         NUM       0.86      0.91      0.88      1734
        PART       0.95      0.77      0.85      5125
        PRON       0.86      0.89      0.87      7444
       PROPN       0.83      0.66      0.74      5473
       PUNCT       1.00      1.00      1.00     29186
       SCONJ       0.75      0.97      0.85      2865
         SYM       1.00      0.85      0.92        62
        VERB       0.97     

Данные в виде таблицы.

In [9]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [10]:
result_model = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'f1_score': f1_scores})
result_model.sort_values('f1_score', ascending=False)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(1, 5))",0.937951
1,"TfidfVectorizer(analyzer='char', ngram_range=(1, 5))",0.924733
8,"HashingVectorizer(analyzer='char', n_features=5000, ngram_range=(1, 5))",0.909227
7,"HashingVectorizer(analyzer='char', n_features=3000, ngram_range=(1, 5))",0.899173
6,"HashingVectorizer(analyzer='char', n_features=2000, ngram_range=(1, 5))",0.891962
2,"HashingVectorizer(analyzer='char', n_features=1000, ngram_range=(1, 5))",0.864823
3,"CountVectorizer(ngram_range=(1, 5))",0.668761
4,"TfidfVectorizer(ngram_range=(1, 5))",0.646806
11,"HashingVectorizer(n_features=5000, ngram_range=(1, 5))",0.585157
10,"HashingVectorizer(n_features=3000, ngram_range=(1, 5))",0.565790


In [11]:
result_model_acc = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'Accuracy': accuracy_scores})
result_model_acc.sort_values('Accuracy', ascending=False)

,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(1, 5))",0.939456
1,"TfidfVectorizer(analyzer='char', ngram_range=(1, 5))",0.927606
8,"HashingVectorizer(analyzer='char', n_features=5000, ngram_range=(1, 5))",0.912403
7,"HashingVectorizer(analyzer='char', n_features=3000, ngram_range=(1, 5))",0.902904
6,"HashingVectorizer(analyzer='char', n_features=2000, ngram_range=(1, 5))",0.895540
2,"HashingVectorizer(analyzer='char', n_features=1000, ngram_range=(1, 5))",0.868644
3,"CountVectorizer(ngram_range=(1, 5))",0.659340
4,"TfidfVectorizer(ngram_range=(1, 5))",0.636402
11,"HashingVectorizer(n_features=5000, ngram_range=(1, 5))",0.603887
10,"HashingVectorizer(n_features=3000, ngram_range=(1, 5))",0.589661


Задание 2.

NLTK

In [12]:
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package tagsets to /home/dmitriy/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/dmitriy/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/dmitriy/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/dmitriy/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [13]:
nltk.help.upenn_tagset('RB')
nltk.help.upenn_tagset('NN')
nltk.help.upenn_tagset('VB')

RB: adverb
    occasionally unabatingly maddeningly adventurously professedly
    stirringly prominently technologically magisterially predominately
    swiftly fiscally pitilessly ...
NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...
VB: verb, base form
    ask assemble assess assign assume atone attention avoid bake balkanize
    bank begin behold believe bend benefit bevel beware bless boil bomb
    boost brace break bring broil brush build ...


In [14]:
records = load_ne5(os.path.join(data_path, r'Collection5/'))

In [15]:
document = next(records)
text = document.text
text = re.sub('\r\n\r\n',' ',text)
text = re.sub('\r\n',' ',text)

In [16]:
document.text = text

In [17]:
nltk.pos_tag(nltk.word_tokenize(text))

[('Валентина', 'JJ'),
 ('Матвиенко', 'NNP'),
 ('стала', 'NNP'),
 ('сенатором', 'NNP'),
 ('Бывший', 'NNP'),
 ('губернатор', 'NNP'),
 ('Санкт-Петербурга', 'JJ'),
 ('Валентина', 'NNP'),
 ('Матвиенко', 'NNP'),
 ('вошла', 'NNP'),
 ('в', 'NNP'),
 ('состав', 'NNP'),
 ('Совета', 'NNP'),
 ('Федерации', 'NNP'),
 (',', ','),
 ('сообщается', 'NNP'),
 ('на', 'NNP'),
 ('сайте', 'NNP'),
 ('администрации', 'NNP'),
 ('Петербурга', 'NNP'),
 ('.', '.'),
 ('Матвиенко', 'VB'),
 ('назначена', 'JJ'),
 ('по', 'NNP'),
 ('указу', 'NNP'),
 ('губернатора', 'NNP'),
 ('Петербурга', 'NNP'),
 ('Георгия', 'NNP'),
 ('Полтавченко', 'NNP'),
 ('как', 'NNP'),
 ('представитель', 'NNP'),
 ('города', 'NNP'),
 ('от', 'NNP'),
 ('исполнительной', 'NNP'),
 ('власти', 'NNP'),
 ('.', '.'),
 ('Постановление', 'VB'),
 ('подписано', 'JJ'),
 ('в', 'NNP'),
 ('среду', 'NNP'),
 (',', ','),
 ('31', 'CD'),
 ('августа', 'NN'),
 (',', ','),
 ('оно', 'NNP'),
 ('вступило', 'NNP'),
 ('в', 'NNP'),
 ('силу', 'NNP'),
 ('в', 'NNP'),
 ('день', 'NNP')

In [18]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))) if hasattr(chunk, 'label') }

{('Бывший', 'PERSON'),
 ('Валентина', 'PERSON'),
 ('Валентина Матвиенко', 'PERSON'),
 ('Георгия Полтавченко', 'PERSON'),
 ('Дмитрий Медведев', 'PERSON'),
 ('Матвиенко', 'ORGANIZATION'),
 ('Матвиенко', 'PERSON'),
 ('Сергея Миронова', 'PERSON'),
 ('Совета Федерации', 'PERSON'),
 ('Совете Федерации', 'PERSON')}

In [19]:
document.spans

[Ne5Span(
     index='T1',
     type='PER',
     start=0,
     stop=19,
     text='Валентина Матвиенко'
 ),
 Ne5Span(
     index='T2',
     type='LOC',
     start=57,
     stop=73,
     text='Санкт-Петербурга'
 ),
 Ne5Span(
     index='T3',
     type='PER',
     start=74,
     stop=93,
     text='Валентина Матвиенко'
 ),
 Ne5Span(
     index='T4',
     type='ORG',
     start=109,
     stop=125,
     text='Совета Федерации'
 ),
 Ne5Span(
     index='T5',
     type='LOC',
     start=161,
     stop=171,
     text='Петербурга'
 ),
 Ne5Span(
     index='T6',
     type='PER',
     start=176,
     stop=185,
     text='Матвиенко'
 ),
 Ne5Span(
     index='T7',
     type='LOC',
     start=217,
     stop=227,
     text='Петербурга'
 ),
 Ne5Span(
     index='T8',
     type='PER',
     start=228,
     stop=247,
     text='Георгия Полтавченко'
 ),
 Ne5Span(
     index='T9',
     type='PER',
     start=386,
     stop=397,
     text='Полтавченко'
 ),
 Ne5Span(
     index='T10',
     type='PER',
     

Встречаются ошибки.

SpaCy

In [20]:
nlp = spacy.load("ru_core_news_sm")

In [21]:
ny_bb = text
article = nlp(ny_bb)

In [22]:
displacy.render(article, jupyter=True, style='ent')

Работает без ошибок

In [23]:
for token in article:
    print(token.text, token.pos_, token.dep_)

Валентина PROPN nsubj
Матвиенко PROPN appos
стала VERB ROOT
сенатором NOUN xcomp
Бывший ADJ amod
губернатор NOUN nsubj
Санкт PROPN nmod
- PROPN nmod
Петербурга PROPN nmod
Валентина PROPN appos
Матвиенко PROPN flat:name
вошла VERB conj
в ADP case
состав NOUN obl
Совета PROPN nmod
Федерации PROPN nmod
, PUNCT punct
сообщается VERB parataxis
на ADP case
сайте NOUN obl
администрации NOUN nmod
Петербурга PROPN nmod
. PUNCT punct
Матвиенко PROPN nsubj:pass
назначена VERB ROOT
по ADP case
указу NOUN obl
губернатора NOUN nmod
Петербурга PROPN nmod
Георгия PROPN appos
Полтавченко PROPN flat:name
как SCONJ case
представитель NOUN obl
города NOUN nmod
от ADP case
исполнительной ADJ amod
власти NOUN nmod
. PUNCT punct
Постановление NOUN nsubj:pass
подписано VERB ROOT
в ADP case
среду NOUN obl
, PUNCT punct
31 ADJ nmod
августа NOUN flat
, PUNCT punct
оно PRON nsubj
вступило VERB conj
в ADP case
силу NOUN obl
в ADP case
день NOUN obl
подписания NOUN nmod
. PUNCT punct
Полтавченко PROPN nsubj
подписа

Свой NER.

In [24]:
def get_classification_report(y_test_true, y_test_pred):
    print(classification_report(y_test_true, y_test_pred))

    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

In [25]:
records = load_ne5(os.path.join(data_path, r'Collection5/'))

In [26]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [27]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [28]:
df_words['tag'].value_counts()

tag
OUT         219214
PER          21200
ORG          13651
LOC           4568
GEOPOLIT      4356
MEDIA         2482
Name: count, dtype: int64

In [29]:
df_words.head()

,word,tag
0,Валентина,PER
1,Матвиенко,PER
2,стала,OUT
3,сенатором,OUT
4,Бывший,OUT


In [30]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

Кодировка целевой переменной.

In [31]:
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [32]:
train_x.apply(len).max(axis=0)

32

In [33]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))
train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

2023-12-17 00:39:06.790849: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1636 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:03:00.0, compute capability: 7.5


In [34]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [35]:
def custom_standardization(input_data):
    return input_data


vocab_size = 30000
seq_len = 10
vectorize_layer = TextVectorization(  
                            standardize=custom_standardization,
                            max_tokens=vocab_size,
                            output_mode='int',
                            #ngrams=(1, 3),
                            output_sequence_length=seq_len)
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [36]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax') # [OUT, PER, ORG, LOC, GEOPOLIT, MEDIA]

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [37]:
mmodel = modelNER()

In [38]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [39]:
mmodel.fit( train_data,
            validation_data=valid_data,
            epochs=3)

Epoch 1/3


2023-12-17 00:39:24.486328: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-17 00:39:26.115258: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f869009e040 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-17 00:39:26.115305: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2023-12-17 00:39:26.131091: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-17 00:39:26.844675: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2023-12-17 00:39:26.956309: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


12444/12444 [==============================] - 103s 8ms/step - loss: 0.2941 - accuracy: 0.9145 - val_loss: 0.2130 - val_accuracy: 0.9378
Epoch 2/3
12444/12444 [==============================] - 97s 8ms/step - loss: 0.1256 - accuracy: 0.9630 - val_loss: 0.4198 - val_accuracy: 0.8935
Epoch 3/3
12444/12444 [==============================] - 99s 8ms/step - loss: 0.1098 - accuracy: 0.9658 - val_loss: 0.3140 - val_accuracy: 0.8934


In [40]:
pred_y = mmodel.predict(valid_x)
y_pred_classes = np.argmax(pred_y,axis=1)

2074/2074 [==============================] - 5s 2ms/step


In [41]:
f1 = f1_score(valid_y, y_pred_classes, average= "weighted")
f1

0.9005803397253274

In [42]:
print(f"Classes: {encoder.classes_}\r\n")

get_classification_report(valid_y, y_pred_classes)

Classes: ['GEOPOLIT' 'LOC' 'MEDIA' 'ORG' 'OUT' 'PER']

              precision    recall  f1-score   support

           0       0.90      0.89      0.90      1082
           1       0.86      0.78      0.82      1144
           2       0.93      0.75      0.83       643
           3       0.86      0.57      0.68      3463
           4       0.97      0.92      0.94     54762
           5       0.49      0.88      0.63      5274

    accuracy                           0.89     66368
   macro avg       0.83      0.80      0.80     66368
weighted avg       0.92      0.89      0.90     66368

CONFUSION MATRIX

col_0    0    1    2     3      4     5
row_0                                  
0      963   23    0    33      8    55
1       12  893    4    26     31   178
2        5    9  485    26     56    62
3       68   42   20  1961    985   387
4       16   74   12   239  50331  4090
5        1    3    0     3    606  4661


Обучение на биграммах и триграммах.

In [43]:
def custom_standardization(input_data):
    return input_data


vocab_size = 30000
seq_len = 10
vectorize_layer = TextVectorization( 
                            standardize=custom_standardization,
                            max_tokens=vocab_size,
                            output_mode='int',
                            ngrams=(1, 3),
                            output_sequence_length=seq_len)
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [44]:
mmodel = modelNER()

In [45]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [46]:
mmodel.fit(train_data,
            validation_data=valid_data,
            epochs=3)

Epoch 1/3


12444/12444 [==============================] - 100s 8ms/step - loss: 0.2970 - accuracy: 0.9135 - val_loss: 0.2135 - val_accuracy: 0.9380
Epoch 2/3
12444/12444 [==============================] - 90s 7ms/step - loss: 0.1267 - accuracy: 0.9627 - val_loss: 0.2898 - val_accuracy: 0.8820
Epoch 3/3
12444/12444 [==============================] - 90s 7ms/step - loss: 0.1106 - accuracy: 0.9656 - val_loss: 0.3476 - val_accuracy: 0.8829


In [47]:
pred_y = mmodel.predict(valid_x)
y_pred_classes = np.argmax(pred_y,axis=1)

2074/2074 [==============================] - 5s 3ms/step


In [48]:
f1 = f1_score(valid_y, y_pred_classes, average= "weighted")
f1

0.906345489697992

In [49]:
print(f"Classes: {encoder.classes_}\r\n")
get_classification_report(valid_y, y_pred_classes)

Classes: ['GEOPOLIT' 'LOC' 'MEDIA' 'ORG' 'OUT' 'PER']

              precision    recall  f1-score   support

           0       0.90      0.89      0.90      1082
           1       0.16      0.93      0.28      1144
           2       0.94      0.76      0.84       643
           3       0.86      0.56      0.68      3463
           4       0.97      0.92      0.94     54762
           5       0.98      0.71      0.82      5274

    accuracy                           0.88     66368
   macro avg       0.80      0.80      0.74     66368
weighted avg       0.95      0.88      0.91     66368

CONFUSION MATRIX

col_0    0     1    2     3      4     5
row_0                                   
0      962    76    0    36      8     0
1       18  1069    0    24     32     1
2        5    69  490    20     57     2
3       66   408   22  1955    993    19
4       14  4075   12   223  50393    45
5        1   884    0     4    660  3725


Результат лучше у сети, которая обучалась на N-граммах